**NOTE: See the Assignments page in the "Start here" module for the rubric that will be used to grade labs.**

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

For this lab we will use the Scikit-Learn [breast cancer](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset) toy dataset. It has various attributes of cell nuclei as features.

In [2]:
(X, y) = load_breast_cancer(return_X_y=True)
m = X.shape[0]
n_features = X.shape[1]
print("Samples: %d\tFeatures: %d"%(m,n_features))

Samples: 569	Features: 30


**1. Use the Scikit-Learn `train_test_split()` function to split X and y into train and test sets, with 80% of the data as the training set, and store in `X_train`, `X_test`, `y_train`, `y_test`.**
  - Hint: check out the examples in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
  - Hint: look at the arguments `test_size` and `train_size`.

In [3]:
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

**2. Fit a [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to just the training input. Use the fit model to scale both the training and testing inputs, storing the scaled data back in `X_train` and `X_test` again.**

In [4]:
# YOUR CODE HERE
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**3. Define a Keras sequential model with two hidden layers, the first with 64 neurons, the second with 32 neurons, and a single output neuron. Use ReLU activation for the two hidden layers and sigmoid for the output layer**
- Hint: remember that the input shape must match the number of features we have.
- Hint: choose appropriate activation functions for each layer

In [6]:
# YOUR CODE HERE
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(n_features,))) # layer 1: 64 neurons, relu activation, input size will match number of features (n_features)
model.add(Dense(32, activation='relu')) # layer 2: 32 neurons, relu activation
model.add(Dense(1, activation='sigmoid')) # layer 3: 1 neuron, sigmoid activation

**4. Compile the model with binary cross-entropy loss, accuracy for metrics, and a Stochastic Gradient Descent optimizer.**

In [7]:
# YOUR CODE HERE
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
# sgd = Stochastic Gradient Descent

**5. Fit the model to the training data, using 16 epochs and a batch size of 8.**

In [8]:
# YOUR CODE HERE
model.fit(X_train, y_train, epochs=16, batch_size=8)

Epoch 1/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8134 - loss: 0.6738
Epoch 2/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.6543
Epoch 3/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8065 - loss: 0.6287
Epoch 4/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8029 - loss: 0.6045
Epoch 5/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8471 - loss: 0.5641
Epoch 6/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8985 - loss: 0.5361
Epoch 7/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9002 - loss: 0.4982
Epoch 8/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9334 - loss: 0.4606
Epoch 9/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9044 - loss: 0.4353
Epoch 10/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9207 - loss: 0.3912
Epoch 11/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9361 - loss: 0.3611
Epoch 12/16
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9319 - lo

## Torch (optional tutorial)

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

Create train/test split and batches

In [10]:
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

ds_train = dataset(X_train, y_train)
dl_train = DataLoader(ds_train, batch_size=16, shuffle=True)

ds_test = dataset(X_test, y_test)
dl_test = DataLoader(ds_test, batch_size=16, shuffle=True)

### Define the model

In the `Week1Network` constructor, we'll create `self.layer1` and `self.layer2` as linear layers. In `forward()`, we'll pass values from layer1 to this layer2, then to a ReLU activation, and store in `x`. Note that when defining a layer, `in_features` must match the `out_features` of the previous layer.

In [11]:
class Week1Network(torch.nn.Module):
    def __init__(self, n_features):
        super(Week1Network, self).__init__()
        self.layer1 = torch.nn.Linear(n_features, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.layer_out = torch.nn.Linear(32,1)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        x = self.layer_out(x)
        x = torch.sigmoid(x)

        return x

### Train the model

We'll create an instance of our `Week1Network` class, and store in `model`, then create a Torch SGD optimizer object with a learning rate of 0.01, and store in `opt`. Finally, we'll create a Torch loss function object of the appropriate type for our binary classification problem, and store in `loss_fn`. Note that the initializer takes the number of features as a parameter. This must match the number of features in the breast cancer dataset that is loaded above.

In [12]:
model = Week1Network(n_features)
opt = torch.optim.SGD(model.parameters(),lr=0.1)
loss_fn = torch.nn.BCELoss()

In [13]:
size = len(dl_train.dataset)

for i in range(16): # epochs
    for batch, (X_batch, y_batch) in enumerate(dl_train):
        pred = model(X_batch)
        loss = loss_fn(pred, y_batch.reshape(-1,1))

        # Backpropagation
        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"epoch: {i+1:2d} loss: {loss:>7f}")


epoch:  1 loss: 0.629791
epoch:  2 loss: 0.596878
epoch:  3 loss: 0.519496
epoch:  4 loss: 0.401336
epoch:  5 loss: 0.174306
epoch:  6 loss: 0.098769
epoch:  7 loss: 0.084555
epoch:  8 loss: 0.643045
epoch:  9 loss: 0.030925
epoch: 10 loss: 0.168089
epoch: 11 loss: 0.341667
epoch: 12 loss: 0.038016
epoch: 13 loss: 0.066692
epoch: 14 loss: 0.038799
epoch: 15 loss: 0.026909
epoch: 16 loss: 0.006782


### Testing

In [14]:
num_batches = len(dl_test)
test_loss, correct = 0, 0

with torch.no_grad():
    for X_batch, y_batch in dl_test:
        pred = model(X_batch)
        test_loss += loss_fn(pred, y_batch.reshape(-1,1))
        correct += (pred.reshape(-1).round() == y_batch).sum().item()

test_loss /= num_batches
correct /= len(dl_test.dataset)
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Test Error: 
 Accuracy: 97.4%, Avg loss: 0.063842 

